### TASK 3

In [121]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score

In [122]:
def load_dataset():
    # Read in the .csv file and split train test
    data_frame = pd.read_csv('Mixcancer.csv')
    data = data_frame.to_numpy()

    # shuffle the data set
    np.random.shuffle(data)
    # Strip off the diagnostic column to get the features
    x = data[:, 1:]

    # Normalise the dataset
    x = MinMaxScaler().fit_transform(x)

    # This column becomes the y
    y = data[:, 0]
    y = y.reshape(len(y), 1)

    return x, y

Define activation and loss functions for use in the model

In [123]:
def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x)*(1.0-sigmoid(x))

def cross_entropy(x, y):
    return (-y*(np.log(x)) - (1-y) * np.log(1-x))

def cross_entropy_derivative(x, y):
    return -(y/x - (1-y)/(1-x))


In [124]:

class ANN:
    def __init__(self, learning_rate, epochs):
        self.learning_rate = learning_rate
        self.epochs = epochs

        self.initialize_weights()

    
    def initialize_weights(self):
        # Moving here for 10 fold code clarity
        # Initialize weights and bias
        neuron_count_l1 = 30
        neuron_count_l2 = 1

        self.w1 = np.random.uniform(-1, 1, (neuron_count_l1, neuron_count_l1))
        self.b1 = np.zeros((1, neuron_count_l1))
        self.w2 = np.random.uniform(-1, 1, (neuron_count_l1, neuron_count_l2))
        self.b2 = np.zeros((1, neuron_count_l2))

    def train(self, x, y):
        fold_size = int(x.shape[0] / 10) # 25
        # Create a fold index
        fold_f_measures = []
        for fold in range(10):
            start_index = fold_size * fold
            end_index = start_index + fold_size
            fold_indices = range(start_index, end_index, 1)

            selection_mask = np.zeros(x.shape[0], dtype=bool)
            selection_mask[fold_indices] = True

            x_test = x[selection_mask]
            y_test = y[selection_mask]

            x_train = x[~selection_mask]
            y_train = y[~selection_mask]

            for epoch_index in range(1, self.epochs+1):

                in_1 = x_train@self.w1 + self.b1
                out_1 = sigmoid(in_1)
                in_2 = out_1@self.w2 + self.b2
                out_2 = sigmoid(in_2)

                train_error = cross_entropy(out_2, y_train).mean()

                # Backpropagate using only the training data
                # Layer 2
                dE_dO2 = cross_entropy_derivative(out_2, y_train)
                dO2_dIn2 = sigmoid_derivative(in_2)
                dIn2_dW2 = out_1
                dE_dW2 = (1/x_train.shape[0])*dIn2_dW2.T@(dE_dO2*dO2_dIn2)
                dE_dB2 = (1/x_train.shape[0]) * np.ones([1, len(x_train)])@(dE_dO2*dO2_dIn2)

                # Layer 1
                dIn2_dO1 = self.w2
                dO1_dIn1 = sigmoid_derivative(in_1)
                dIn1_dW1 = x_train
                dE_dW1 = (1/x_train.shape[0])*dIn1_dW1.T@((dE_dO2*dO2_dIn2@dIn2_dO1.T)*dO1_dIn1)
                dE_dB1 = (1/x_train.shape[0])*np.ones([len(x_train)])@((dE_dO2*dO2_dIn2@dIn2_dO1.T)*dO1_dIn1)

                # Updating parameters
                self.b2 -= self.learning_rate * dE_dB2
                self.w2 -= self.learning_rate * dE_dW2
                self.b1 -= self.learning_rate * dE_dB1
                self.w1 -= self.learning_rate * dE_dW1

                if epoch_index % 100 == 0:
                    print("Training loss fold {} at epoch: {} = {}".format(fold + 1, epoch_index, train_error))

            # Get F measure of the test set after model training
            in_1 = x_test@self.w1 + self.b1
            out_1 = sigmoid(in_1)
            in_2 = out_1@self.w2 + self.b2
            out_2 = sigmoid(in_2)

            test_pred = np.where(out_2 > 0.5, 1, 0)
            test_f_measure = f1_score(y_test, test_pred)

            print("F-Measure for fold {} = {}".format(fold + 1, test_f_measure))
            fold_f_measures.append(test_f_measure)

            # Reset the weights and baises at the end of each fold's run
            self.initialize_weights()
        
        # Print average F measure score 
        print("F-Measure average for the 10 folds = {}".format(np.mean(fold_f_measures)))
        print("F-Measure variance = {}".format(np.max(fold_f_measures) - np.min(fold_f_measures)))


In [125]:
def main():
    # load the dataset with applied random shuffle
    x, y = load_dataset()

    ann = ANN(learning_rate=0.008, epochs=3000)

    ann.train(x, y)
main()

Training loss fold 1 at epoch: 100 = 0.5870528093065274
Training loss fold 1 at epoch: 200 = 0.5740949661189264
Training loss fold 1 at epoch: 300 = 0.5618221796267864
Training loss fold 1 at epoch: 400 = 0.5500280095382002
Training loss fold 1 at epoch: 500 = 0.5386744971304253
Training loss fold 1 at epoch: 600 = 0.5277278258013873
Training loss fold 1 at epoch: 700 = 0.5171577654494934
Training loss fold 1 at epoch: 800 = 0.5069376733426735
Training loss fold 1 at epoch: 900 = 0.49704435066573976
Training loss fold 1 at epoch: 1000 = 0.4874577916461908
Training loss fold 1 at epoch: 1100 = 0.47816086481090025
Training loss fold 1 at epoch: 1200 = 0.469138959859572
Training loss fold 1 at epoch: 1300 = 0.46037962680561767
Training loss fold 1 at epoch: 1400 = 0.45187222719624753
Training loss fold 1 at epoch: 1500 = 0.44360761090792805
Training loss fold 1 at epoch: 1600 = 0.4355778266062056
Training loss fold 1 at epoch: 1700 = 0.42777586964869063
Training loss fold 1 at epoch: 1800